In [1]:
import pandas as pd
import requests
import json
import hashlib

In [2]:
payload1 = json.dumps({
         'question': 'What is the price?',
         'context': 'High price, Korean style'
})
res1 = json.loads(requests.post('http://163.14.137.78:1300/getAnswer',data = payload1).text)

In [3]:
res1

{'answer': 'High price', 'end': 10, 'score': 0.5748034119606018, 'start': 0}

In [11]:
payload2 = json.dumps({
         'sentences':[['','好的'],['还在吗','请问还在吗']]
})
res2 = json.loads(requests.post('http://127.0.0.1:5001/getAnswer',data = payload2).text)
print(res2)

{'content': "[['0', 0.59274155], ['1', 0.9808791]]"}


In [4]:
# script = {"Stage":[],"ID":[],"Input":[],"Output":[],"Context_Type":[]}

In [5]:
#script_df = pd.DataFrame(script)
script_df = pd.read_csv('script.csv')

In [6]:
# script_df.to_csv('script.csv',index=False)

In [ ]:
dialogs = {}

In [8]:
# 語義匹配，需要威宗使用NLP來實現高級功能，return str
def CheckInput(q):
    return 'checked'

In [9]:
# 檢查Context_Type，需要威宗使用NLP來實現高級功能，return str
def CheckType(q):
    if q == "你好":
        return "Greeting"
    if q == "餐廳":
        return "Shop"
    if q == "停下":
        return "Stop"

In [10]:
# 負責最終的推薦
def Rcmd():
    return 'test_shop'

In [11]:
#此方法最終只輸出一個str
def dialog(q,uid):
    try:
        current_dialog = dialogs[uid]
    except:
        dialogs[uid] = {'stage':0,'store_features':[]}
        return '你好，似乎是初次见面，请问需要什么帮助吗？'
    # 檢測打招呼
    # if stage == 0 and CheckType(q) == "Greeting":
    #     session = script_df[script_df.Input == q]
    #     return session.iloc[0]['Output']
    # 詢問店家就這麼做
    if CheckType(q) == "Shop":
        current_dialog['stage'] = 1
        session = script_df[script_df.Input == q]
        return session.iloc[0]['Output']
    # 詢問商店特徵就這麼做
    if current_dialog['stage'] == 1 and CheckType(q) != 'Stop':
        # 先寫入商店特徵
        current_dialog['store_features'].append(q)
        return '收到！請問還要補充嗎？'
    if current_dialog['stage'] == 1 and CheckType(q) == 'Stop':
        # 先寫入商店特徵
        current_dialog['stage'] = 2
        return 'OK，現在幫你推薦！'
    if current_dialog['stage'] == 2:
        return '不好意思，請問想重新推薦嗎？'

In [12]:
# dialog('你好')
# dialog('餐廳')
# dialog('高價')
# dialog('韓式')
# dialog('停下')

'你好，請問需要什麼幫助：餐廳or餐點？'

In [17]:
def translate_z2e(query):
    appid = '20220102001044938'
    q = query
    salt='123456'
    key = '4i3lYa4aSyuH2TCcmaBh'
    mid = appid + q + salt + key
    hl = hashlib.md5()
    hl.update(mid.encode(encoding='utf-8'))
    sign = hl.hexdigest()
    url = f'https://api.fanyi.baidu.com/api/trans/vip/translate?q={q}&from=zh&to=en&appid={appid}&salt={salt}&sign={sign}'
    return eval(requests.get(url).text)['trans_result'][0]['dst']

In [18]:
def translate_e2z(query):
    appid = '20220102001044938'
    q = query
    salt='123456'
    key = '4i3lYa4aSyuH2TCcmaBh'
    mid = appid + q + salt + key
    hl = hashlib.md5()
    hl.update(mid.encode(encoding='utf-8'))
    sign = hl.hexdigest()
    url = f'https://api.fanyi.baidu.com/api/trans/vip/translate?q={q}&from=en&to=zh&appid={appid}&salt={salt}&sign={sign}'
    return eval(requests.get(url).text)['trans_result'][0]['dst']